# RNN - LSTM을 이용한 Hiphop 가사 만들기
-------

자신의 진솔한 이야기를 간결하게 담아낸 Hiphop 가사.

많은 가사를 인공지능으로 학습시켜 새로운 가사를 만들어내자!

**주제 선정 이유**
- 국어국문학과 출신으로 머신러닝, 딥러닝을 배우며 컴퓨터 언어로 자연어를 처리하는 것의 매력을 느꼈다
- 현실 세계의 언어를 학습시켜 새로운 문장을 만들어내는 것을 프로젝트 주제로 선정하여, 얼마나 잘 학습이 가능한지, 어떤 방법을 취하는 것이 효율적인지 알아보고자 한다

> **1. 목표**

> - Rap/Hiphop 장르의 가사들을 학습시켜 새로운 가사를 생성해내고, **말이 되게끔** 만든다


> **2. 분석 내용**

> - 웹 크롤링, DB 관리
> - n-gram / Word2Vec / t-SNE 를 통한 자연어 처리
> - 딥러닝의 RNN - LSTM 알고리즘으로 가사 학습 및 생성
> - Flask 모듈, html 활용을 통한 웹페이지 구현


> **3. Workflow**

> 1) Data 수집
>    - Hiphop 가사를 웹 크롤링 (각종 문장부호 함께 제거)
>    - MySQL에 저장
>
> 2) 워드클라우드
>    - Hiphop 장르 9000여 곡의 가사를 명사화시켜 워드클라우드 생성
>    - 좋아요 기준 상위 1000곡의 인기곡 가사를 명사화시켜 워드클라우드 생성
        
> 3) 전처리
>    - 자주 나오는 문구를 엮어 bigram 처리
        
> 4) Word2Vec (Skip-gram 방식)
>    - bigram 처리한 텍스트의 단어들을 vector화

> 5) t-SNE
>    - Word2Vec 처리한 단어 중 100회 이상 등장하는 단어들의 유사도를 찾아 시각화

> 6) RNN - LSTM으로 가사 생성

> 7) Python Flask를 이용한 웹페이지 구현

In [61]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import pandas as pd
import MySQLdb
import sklearn

import konlpy
from konlpy.tag import Twitter

import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim import corpora, models
from gensim.models import LdaMulticore
from gensim.models import Word2Vec
from gensim.corpora import Dictionary, MmCorpus

import pyLDAvis
import pyLDAvis.gensim
import pickle

from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

# from nltk.tokenize import RegexpTokenizer
from os import path
# from itertools import islice

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value
output_notebook()

Loading BokehJS ...

## 1. Data 수집
- 국내 최대 음원사이트 Melon에서 Rap/Hiphp 장르의 전체 곡을 selenium을 이용해 크롤링

### 1) 가사 웹 크롤링

1-1. 노래 1곡의 제목, 좋아요, 가수, 가사 크롤링

In [ ]:
driver = webdriver.Chrome()
driver.get("http://www.melon.com/genre/song_list.htm?gnrCode=GN0200")


lyric_page = driver.find_element_by_xpath("//a[@class='btn btn_icon_detail']")
lyric_page.click()

title = driver.find_element_by_xpath("//p[@class='songname']")
print ("제목 :",title.text)

likes = driver.find_element_by_xpath("//span[@id='d_like_count']")
print ("좋아요 :",likes.text)

singer = driver.find_element_by_xpath("//a[@class='atistname']")
print ("가수 :",singer.text)

lyrics = driver.find_element_by_xpath("//div[@class='lyric']")
print ("가사 :")
print (lyrics.text)

1-2. url 페이지를 넘기며 크롤러 완성

In [ ]:
driver = webdriver.Chrome()
for i in range(10):
    i = i * 50 + 1
    url = "http://www.melon.com/genre/song_list.htm?gnrCode=GN0300#params%5BgnrCode%5D=GN0300&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&po=pageObj&startIndex={}".format(i)
    driver.get(url)
    for i in range(50):
        try:
            element = driver.find_elements_by_xpath("//a[@class='btn btn_icon_detail']")
            element[i].click()
            title = driver.find_element_by_xpath("//p[@class='songname']")
            print ("Title :",title.text)
            likes = driver.find_element_by_xpath("//span[@id='d_like_count']")
            print ("Likes :",likes.text)
            singer = driver.find_element_by_xpath("//a[@class='atistname']")
            print ("Singer :",singer.text)
            lyrics = driver.find_element_by_xpath("//div[@class='lyric']")
            print ("Lyrics :")
            print (lyrics.text)
            driver.back()
            i += 1
        except:
            driver.back()
            i += 1
    driver.back()
    i += 1 

### 2) MySQL DB에 저장 - 총 9139곡

2-1. 1곡만 넣어보기

In [ ]:
server = ''
db_id = ''
db_pwd = ''
db_db = ''

In [ ]:
db = MySQLdb.connect(server, db_id, db_pwd, db_db)
db.set_character_set('utf8') # 불러올 때 utf8로 선언
cursor = db.cursor()

driver = webdriver.Chrome()
driver.get('http://www.melon.com/song/detail.htm?songId=30568338')

singer = driver.find_element_by_xpath("//a[@class='atistname']")
title = driver.find_element_by_xpath("//p[@class='songname']")
likes = driver.find_element_by_xpath("//span[@id='d_like_count']")
lyrics = driver.find_element_by_xpath("//div[@class='lyric']")

try:
    query = """INSERT INTO lyrics
                    VALUES ('{}', '{}', '{}', '{}')""" \
                    .format(singer.text.replace("'", '').replace('"', ''), title.text.replace("'", '').replace('"', ''), int(likes.text.replace(',', '')), lyrics.text.replace("'", '').replace('"', ''))
    cursor.execute(query)

    db.commit()
except Exception as e:
    db.rollback()
    print (e )

cursor.close()
    
# disconnect from server
db.close()

2-2. for문을 돌려 전체 저장

In [ ]:
db = MySQLdb.connect(server, db_id, db_pwd, db_db)
db.set_character_set('utf8')
cursor = db.cursor()

driver = webdriver.Chrome()
for a in range(271):
    a = a * 50 + 1
    url = "http://www.melon.com/genre/song_list.htm?gnrCode=GN0300#params%5BgnrCode%5D=GN0300&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=NEW&po=pageObj&startIndex={}".format(a)
    driver.get(url)
    for i in range(50):
        try:
            element = driver.find_elements_by_xpath("//a[@class='btn btn_icon_detail']")
            element[i].click()
            title = driver.find_element_by_xpath("//p[@class='songname']")
            likes = driver.find_element_by_xpath("//span[@id='d_like_count']")
            singer = driver.find_element_by_xpath("//a[@class='atistname']")
            lyrics = driver.find_element_by_xpath("//div[@class='lyric']")
            try:
                query = """INSERT INTO lyrics
                    VALUES ('{}', '{}', '{}', '{}')""" \
                    .format(singer.text.replace("'", '').replace('"', ''), title.text.replace("'", '').replace('"', ''), int(likes.text.replace(',', '')), lyrics.text.replace(".", '').replace("*", '').replace("-", '').replace(",", '').replace("'", '').replace('"', ''))
                cursor.execute(query)
                db.commit()
            except Exception as e:
                db.rollback()
                print (e)
            driver.back()
            i += 1
        except:
            driver.back()
            i += 1
    driver.back()
    print(a)
    a += 1
    
print("DONE")

db.close()

> - selenium을 이용하여 직접 웹 크롤링을 실행하였고, MySQL과 Workbench를 사용함으로써 DB 사용 및 SQL 구문에 익숙해질 수 있었다.

## 2. 워드클라우드

Hiphop 장르 전체의 가사 가져오기

In [3]:
with open('data/hiphop.txt', 'r', encoding = 'utf-8') as f:
    hiphop_txt = f.read()

In [4]:
print (hiphop_txt[:100])

﻿women women women
women women women
why are you so sad why are you
너무 지쳐보여 잠시 내려놔도 돼
여긴 널 위한 paradi


좋아요 상위 1000곡의 인기곡 가사 가져오기

In [5]:
with open('data/hiphop_1000.txt', 'r', encoding = 'utf-8') as f:
    hiphop_1000 = f.read()

In [6]:
print(hiphop_1000[:10])

﻿보고 싶다
이렇게


힙합 장르 9139곡의 가사 / 좋아요 상위 1000곡 모두 명사화

In [7]:
tw = Twitter()
stop_words = open('data/stop_words.txt', 'r').read().split('\n')

def normalize(lyric):
    nouns = tw.nouns(lyric)
    lyric_noun = [ noun for noun in nouns if len(noun) > 1 and noun not in stop_words]
    return lyric_noun

In [8]:
%%time
normalized_text = normalize(hiphop_txt)
normalized_1000 = normalize(hiphop_1000)

Wall time: 5min 39s


### 1) 힙합 장르 전체의 가사 워드클라우드

In [9]:
WC = WordCloud('font/Daum_SemiBold.ttf', width=700, height=700)

In [11]:
%%time
hiphop_wc = WC.generate(' '.join(normalized_text))
hiphop_wc.to_file('wordcloud/hiphopwordcloud.png')

Wall time: 14 s


![Hiphop 9100곡 워드클라우드](wordcloud/hiphopwordcloud.png)

이제, 우리, 지금 오늘, 시간, 모두, 사람, 너무 등이 많이 등장했다.

### 2) 인기곡 1000곡의 가사 워드클라우드

In [12]:
%%time
hiphop_wc = WC.generate(' '.join(normalized_1000))
hiphop_wc.to_file('wordcloud/wordcloud1000.png')

Wall time: 3.46 s


![Hiphop 1000곡 워드클라우드](wordcloud/wordcloud1000.png)

우리가 가장 크게 보이며, 지금, 이제, 사랑, 사람, 시간, 너무가 자주 등장한다.

> - 워드클라우드 처리를 통해, 가사에서 자주 등장하는 명사들을 알아볼 수 있었다.
- 또한 9000곡과 인기곡 1000곡 간에 많이 등장하는 단어는 큰 차이가 없는 것이 확인 가능하다.

## 3. 전처리
- RNN 모델에 넣을 텍스트 파일 전처리
- '말이 되는' 가사 생성에 도움받기 위해 자주 나오는 단어들을 묶어 어구로 만들었다
- n-gram 이라는 기법으로 bigram(두 단어씩 묶기), trigram(세 단어씩 묶기) 등의 방법이 있다
- 여기서는 두 단어씩 묶는 bigram을 사용하였다

### 1) 자주 나오는 어구의 bigram 처리

1-1. 9139곡 가사 전체를 가져옴

In [15]:
hiphop = pd.read_csv('data/hiphop.csv')
hiphop.head()

,singer,title,likes,lyrics
0,러버소울 (Rubber Soul),Beautiful Women,91,women women women\nwomen women women\nwhy are ...
1,PD 블루,빠빠야 빠빠,9,이젠 나와 함께할 널 놓치지 않아\n이제는 용기 내겠어\nhello baby in ...
2,Genius D,마주,16,나도 알아 원래부터 너는 예뻤어\n머리부터 발 끝까지 다 완벽했어\n분명 단점이라곤...
3,윤선생,놀러갈까? (Feat. 오요),3,하 지겹다\n아침에 일어나서 얼굴을 봐봐\n미간에 주름 피려 억지로 활짝\n매일 똑...
4,엠케이,최 문 기 와 이 원 미 (W E D D I N G V E R S I O N - F...,1,이건 그동안의 가사와는 다른\n이건 리얼 스토리\n이건 지금 벌어지고 있는 이야기\...


In [16]:
%%time
USE_PREMADE_TEXT = False

text_filepath = 'word2vec/all_lyrics_text.txt'
if not USE_PREMADE_TEXT:
    with open(text_filepath, 'w', encoding='utf-8') as f:
        for lyrics in hiphop.lyrics.values:
            if pd.isnull(lyrics): # null값 있다면 그 다음으로 넘어감
                continue
            f.write(lyrics + '\n')
else:
    assert path.exists(text_filepath)

Wall time: 394 ms


1-2. Phrase modeling - bigram 모델 저장

In [17]:
%%time
USE_PREMADE_BIGRAM_MODEL = False

all_bigram_model_filepath = 'word2vec/all_bigram_model'
all_sentences_normalized_filepath = 'word2vec/all_lyrics_text.txt'

# gensim's LineSentence provies a convenient way to iterate over lines in a text file.
# it outputs one line at a time, so you can save memory space. it works well with other gensim components.

# we take normalized sentences as unigram sentences, which means we didn't apply any phrase modeling yet.
all_unigram_sentences = LineSentence(all_sentences_normalized_filepath)

if not USE_PREMADE_BIGRAM_MODEL:    
    
    all_bigram_model = Phrases(all_unigram_sentences) #phrase냐 아니냐를 판단해줌
    all_bigram_model.save(all_bigram_model_filepath)
    
else:
    all_bigram_model = Phrases.load(all_bigram_model_filepath)

Wall time: 18.9 s


1-3. bigram txt파일 저장

In [19]:
%%time
USE_PREMADE_BIGRAM_SENTENCES = False

all_bigram_sentences_filepath = 'word2vec/all_sentences_for_word2vec.txt'

if not USE_PREMADE_BIGRAM_SENTENCES:
    
    with open(all_bigram_sentences_filepath, 'w', encoding='utf-8') as f:
        for unigram_sentence in all_unigram_sentences:
            all_bigram_sentence = all_bigram_model[unigram_sentence]
            f.write(' '.join(all_bigram_sentence) + '\n')
else:
    assert path.exists(all_bigram_sentences_filepath)

C:\Python3\lib\site-packages\gensim\models\phrases.py:274: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 34.1 s


> - RNN 모델을 학습시킬 때, 말이 되는 어구가 나오도록 만들어주기 위하여 bigram 처리를 하였다.

## 4. Word2Vec
- 컴퓨터가 자연어를 인식하려면 텍스트 그대로가 아닌, '숫자화'된 언어를 입력시켜야 한다
- Word2Vec 는 텍스트 안의 각 단어를 벡터화시킨다
- 또한 각 벡터에 의미를 담아 비슷한 의미의 단어가 비슷한 벡터로 표현되도록 만든다
- bigram 처리한 텍스트로 Word2Vec 모델을 만들어, RNN에 함께 학습시켰다

### 1) Word2Vec으로 자연어를 vector 처리

1-1. bigram 처리한 Hiphop 장르 곡 전체를 word2vec

In [21]:
%%time
USE_PREMADE_WORD2VEC = False

all2vec_filepath = 'word2vec/all_word2vec_model'

if not USE_PREMADE_WORD2VEC:
    
    lyrics_for_word2vec = LineSentence(all_bigram_sentences_filepath)

    all2vec = Word2Vec(lyrics_for_word2vec, size=100, window=5, min_count=1, sg=1)
    # sg=0 cbow 1=Skip-Gram Model
    # 100차원으로 가져옴 / 보통 20~100 정도
    # window = 5 앞5개, 뒤5개 // 가장 효과가 큰 게 window size임!!!!
    # window size 작을수록 문법적인 의미가 너무 중요해짐, 클수록 주제 지향적으로 문맥적인 정보를 많이 담게 됨    
    for _ in range(9):
        all2vec.train(lyrics_for_word2vec, total_examples=789847, epochs=1)
        # 문장 총 789847

    all2vec.save(all2vec_filepath)
else:
    all2vec = Word2Vec.load(all2vec_filepath)
all2vec.init_sims()

Wall time: 3min 41s


1-2. 저장한 모델 불러오기

In [22]:
all2vec_filepath = 'word2vec/all_word2vec_model'
all2vec = Word2Vec.load(all2vec_filepath)

1-3. Word2Vec 내용 확인

In [99]:
all2vec.most_similar(positive=['사랑'], topn=5)

[('이별', 0.6620835661888123),
 ('고백', 0.6474951505661011),
 ('연애', 0.6245040893554688),
 ('되갚을', 0.6115529537200928),
 ('우정', 0.5924676656723022)]

In [98]:
all2vec.most_similar(positive=['첫번째'], topn=5)

[('최선의', 0.7836522459983826),
 ('유일한', 0.7827488780021667),
 ('신이_주신', 0.778355598449707),
 ('단_하나의', 0.7782732248306274),
 ('LP와', 0.7718019485473633)]

> - Word2Vec 에서는 한 단어의 앞뒤로 각각 5개, 총 10개의 단어를 보도록 했으며, 1번이라도 등장한 단어까지 모두 포함시켰다.
- 모델 생성 시 Skip-gram 방식을 사용했는데, 현재 주어진 단어 하나를 가지고 주위에 등장하는 나머지 단어들의 등장 여부를 유추하는 방법이다.
- 위 코드는 각 단어와 유사도가 높은 다른 단어를 순서대로 정렬한 것이다. '사랑'과 '이별', '고백', '연애' 등이 높은 수치로 가깝게 있음을 알 수 있다.
- '첫번째' 라는 단어는 '최선의', '유일한', '신이 주신', '단 하나의' 등과 유사하게 나타난다.
- 이와 같이 Word2Vec 기법은 단어를 벡터화하며 유사한 단어를 최대한 유사한 벡터값으로 처리해준다.

## 5. t-SNE
- t-SNE란 차원축소 기법의 하나로 시각화를 위해 주로 사용된다
- t-SNE는 고차원에서 가까운 포인트를 저차원에서도 계속 가깝게 두고, 먼 포인트는 계속 멀게 유지시킨다
- 데이터간의 유사도를 통해 데이터의 패턴을 찾을 수 있다
- 여기서는 9000여 곡 가사 단어 중 100회 초과 등장한 단어들을 대상으로, word2vec로 얻어진 각 단어의 numpy 행렬값을 매핑시킨 후 2차원 그래프로 시각화하였다.

5-1. word2vec 모델링을 통해 얻어진 단어는 총 278901개

In [102]:
a = pd.DataFrame(all2vec.wv.index2word)
a.head()

,0
0,난
1,내
2,내가
3,다
4,you


In [101]:
len (a)

278901

5-2. 이 중 100회 이상 등장한 단어들만 리스트화

In [70]:
%%time
words = []
for i in (range(278901)):
    cnt = all2vec.wv.vocab[a[0][i]] # 횟수 카운트
    if cnt.count > 100:
        words.append(a[0][i]) # 횟수 100회 초과하는 단어만 리스트화
    i += 1

Wall time: 10.2 s


가사 전체에서 100회 초과 등장하는 단어는 총 2876개

In [89]:
len (words)

2876

5-3. 100회 이상 등장한 단어들에 대해 vector 값 가져옴

In [90]:
X = []
for i in words:
    X.append(all2vec.wv[i])

In [91]:
type (X)

list

5-4. 단어와 각 vector 값으로 데이터 프레임 생성

In [92]:
X2 = pd.DataFrame(X, index = words)

In [93]:
X2.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
난,0.458440,0.557801,-0.333001,1.032216,0.058768,0.170677,0.228761,-0.139880,0.299976,0.576202,...,-1.126730,-0.229988,-0.278105,-0.545295,0.170282,0.829141,-0.033361,0.304362,0.963970,0.510931
내,0.110845,0.350505,-0.221804,0.237350,0.064332,0.006314,0.616054,0.294108,0.444770,-0.558680,...,-0.536083,0.086402,-0.160988,0.277951,0.167073,0.487603,-0.166118,0.467342,-0.108386,-0.108666
내가,-1.103762,-0.161733,-0.241357,0.283349,-0.334328,0.089152,0.549801,0.834607,0.116966,0.049266,...,-0.703710,0.412801,0.607993,-0.473825,-0.536843,-0.316118,0.487226,-0.925775,-0.244928,1.271176
다,-1.013392,-0.090737,-0.933017,1.093223,-0.205201,0.016055,1.284712,-1.158620,0.457222,-0.611903,...,-0.848001,-0.842088,0.441641,-0.791193,0.284579,-1.309415,0.975858,-0.378933,-0.519747,0.948633
you,-1.077578,0.758905,-1.519394,-0.414121,-0.136386,0.548430,0.356182,-0.289389,0.126165,0.971461,...,-1.378384,-1.193813,-0.288117,0.135336,0.321420,-0.082028,-0.362976,-0.410056,0.228079,0.809649


5-4. t-SNE 모델링

In [103]:
%%time
USE_PREMADE_TSNE = False

tsne_filepath = 'Example/tsne.pkl'

if not USE_PREMADE_TSNE:
    
    tsne = TSNE(random_state=0)
    tsne_points = tsne.fit_transform(X2)
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne_points, f)
else:
    with open(tsne_filepath, 'rb') as f:
        tsne_points = pickle.load(f)

tsne_df = pd.DataFrame(tsne_points, index=X2.index, columns=['x_coord', 'y_coord'])
tsne_df['word'] = tsne_df.index

Wall time: 1min 11s


5-5. t-SNE 모델 시각화

In [104]:
# prepare the data in a form suitable for bokeh.
plot_data = ColumnDataSource(tsne_df)

# create the plot and configure it
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   active_scroll='wheel_zoom'
                  )

# add a hover tool to display words on roll-over
tsne_plot.add_tools( HoverTool(tooltips = '@word') )

tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='red', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='orange')

# adjust visual elements of the plot
tsne_plot.title.text_font_size = value('16pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show time!
show(tsne_plot);

> - 각 데이터 포인트에 마우스를 올리면 어떤 단어들이 함께 맵핑되어 있는지 확인 가능하다. 특히 진한 부분일 수록 비슷한 단어들이 많이 모여 있음을 알 수 있다.
- [꽤, 뭔가, 생각보다] 와 같이 단어 자체는 다르지만 의미가 비슷한 것들이 함께 매칭되어 있다.
- [됐지, 된, 되는, 됐어] 와 같이 '되다' 에서 파생된 단어들이 함께 모여있다.
- [귀를, 손을, 손과] 와 같이 단어의 모양과 뜻 모두 다르지만 신체 부위를 일컫는 공통점이 있는 포인트들도 있다.
- 이것으로 보아 텍스트의 Word2Vec 처리와 t-SNE 모델링이 우수하게 이뤄진 것이 확인 가능하다.

## 6. RNN - LSTM으로 가사 생성
- 일반적인 뉴럴넷에 비해 RNN은 파라미터가 공유되며, backpropagation through time 이라는 특징이 있다
- 하지만 기존의 RNN은 긴 시퀀스를 잘 처리하지 못하며, 그라디언트가 소멸되는 현상이 발생함
- LSTM이라는 RNN의 변형 모델이 등장
- LSTM은 '게이트'를 통해 상태값을 계산 시 내부의 상태값을 얼마나 기억하고 사용할지 정한다
- 그라디언트 흐름을 좋게 만들고 베니싱 현상을 완화시켜 먼 과거에 대한 단어나 정보도 잘 기억할 수 있게 되었다

### 1) 1000곡의 가사로 학습시킴
- Hiphop장르 1000곡의 가사를 RNN - LSTM 방식으로 학습
- 23만회 학습시킨 모델을 생성
- 초반 단어 3,4개를 입력하여 차후 나올 단어들을 예측

![가사캡쳐](capture/3.png)

![가사캡쳐](capture/4.png)

![가사캡쳐](capture/8.png)

![가사캡쳐](capture/10.png)

### 2) 보완할 점

![가사캡쳐](capture/잘못된_예.png)

> - 자주 나오는 단어를 입력할수록 출력 단어가 자꾸 반복되는 현상을 발견
- 23만회 이상으로 학습시키는 것이 목표
- 차후 Beam Search 등의 알고리즘 적용하여 더 말이 되게끔 만드는 작업이 필요하다!

## 7. Python Flask를 이용한 웹페이지 구현
- Flask 모듈은 웹 개발을 위한 모듈
- Micro-Framework 를 사용함으로써 핵심적인 기능만 자체적으로 지원하고 나머지는 기존에 존재하는 모듈을 사용
- 학습이나 구현에 편리함